In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false'
os.environ['CUDA_VISIBLE_DEVICES']='9'


In [3]:
import jax
import jax.numpy as jnp

In [32]:
def reshape(Q_funcs=[1,2,3,4], levels=[1,3], n_ensemble=3):
   # Q_funcs: jnp.ndarray of size \sum_{i} levels[i]
   # levels: list of numbers 
   # returns Q_matrix = jnp.ndarray of shape [len(n_levels), n_ensemble], where
   # (Q_{matrix}){i, j} = something from Q_funcs if j < levels[i], else 0
    Q_matrix = jnp.zeros((len(levels), n_ensemble))
    i = 0
    for n, level_size in enumerate(levels):
        Q_matrix = Q_matrix.at[n, :level_size].set(Q_funcs[i:i+level_size])
        i = i + level_size
    return Q_matrix

In [50]:
import numpy as np

def create_level_specification(levels, n_ensemble):
    N = np.zeros((len(levels) * n_ensemble, sum(levels) + len(levels)))
    current_level = 0
    current_idx = 0
    for i in range(sum(levels) + len(levels)):
        print(i, current_level, current_idx)
        i_index = current_level * n_ensemble + current_idx
        N[i_index, i] = 1
        current_idx += 1
        if current_idx > levels[current_level]:
            current_idx = 0
            current_level += 1
    return N

In [51]:
level_specification = create_level_specification((0, 2), 3)

0 0 0
1 1 0
2 1 1
3 1 2


In [52]:
level_specification

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [53]:
def reshape(Q, level_specification, n_ensemble):
    Q_reshaped = level_specification.dot(Q)
    return Q_reshaped.reshape(-1, n_ensemble)

In [54]:
reshape(jnp.arange(1, 5), level_specification, n_ensemble=3)

array([[1., 0., 0.],
       [2., 3., 4.]])

In [35]:
tuple(jnp.arange(1, 5))

(DeviceArray(1, dtype=int32),
 DeviceArray(2, dtype=int32),
 DeviceArray(3, dtype=int32),
 DeviceArray(4, dtype=int32))

In [29]:
slice(0, 1)

slice(0, 1, None)

In [57]:
jax.jit(reshape, static_argnums=2)(jnp.arange(1, 5), level_specification, 3)

DeviceArray([[1., 0., 0.],
             [2., 3., 4.]], dtype=float32)

In [40]:
from typing import Any, Optional, Tuple

import flax.linen as nn
import jax.numpy as jnp

from tensorflow_probability.substrates import jax as tfp

tfd = tfp.distributions
tfb = tfp.bijectors


def default_conv_init(scale: Optional[float] = jnp.sqrt(2)):
    return nn.initializers.xavier_uniform()

def default_mlp_init(scale: Optional[float] = 0.01):
    return nn.initializers.orthogonal(scale)

def default_logits_init(scale: Optional[float] = 0.01):
    return nn.initializers.orthogonal(scale)


class ResidualBlock(nn.Module):
    """Residual block."""
    num_channels: int
    prefix: str

    @nn.compact
    def __call__(self, x):
        # Conv branch
        y = nn.relu(x)
        y = nn.Conv(self.num_channels,
                    kernel_size=[3, 3],
                    strides=(1, 1),
                    padding='SAME',
                    kernel_init=default_conv_init(),
                    name=self.prefix + '/conv2d_1')(y)
        y = nn.relu(y)
        y = nn.Conv(self.num_channels,
                    kernel_size=[3, 3],
                    strides=(1, 1),
                    padding='SAME',
                    kernel_init=default_conv_init(),
                    name=self.prefix + '/conv2d_2')(y)

        return y + x


class Impala(nn.Module):
    """IMPALA architecture."""
    prefix: str

    @nn.compact
    def __call__(self, x):
        out = x
        for i, (num_channels, num_blocks) in enumerate([(16, 2), (32, 2),
                                                        (32, 2)]):
            conv = nn.Conv(num_channels,
                           kernel_size=[3, 3],
                           strides=(1, 1),
                           padding='SAME',
                           kernel_init=default_conv_init(),
                           name=self.prefix + '/conv2d_%d' % i)
            out = conv(out)

            out = nn.max_pool(out,
                              window_shape=(3, 3),
                              strides=(2, 2),
                              padding='SAME')
            for j in range(num_blocks):
                block = ResidualBlock(num_channels,
                                      prefix='residual_{}_{}'.format(i, j))
                out = block(out)
            print(out.shape)
        print(out.shape)
        out = out.reshape(out.shape[0], -1)
        out = nn.relu(out)
        out = nn.Dense(256, kernel_init=default_mlp_init(), name=self.prefix + '/representation')(out)
        out = nn.relu(out)
        return out


class TwinHeadModel(nn.Module):
    """Critic+Actor for PPO."""
    action_dim: int
    prefix_critic: str = "critic"
    prefix_actor: str = "policy"

    @nn.compact
    def __call__(self, x):
        z = Impala(prefix='shared_encoder')(x)
        # Linear critic
        v = nn.Dense(1, kernel_init=default_mlp_init(), name=self.prefix_critic + '/fc_v')(z)

        logits = nn.Dense(self.action_dim,
                        kernel_init=default_logits_init(),
                        name=self.prefix_actor + '/fc_pi')(z)

        pi = tfd.Categorical(logits=logits)
        
        return v, pi

In [4]:
import jax

In [44]:
X = jnp.zeros((256, 64, 64, 3))

In [41]:
model = Impala('critic')

In [42]:
key1, key2 = jax.random.split(jax.random.PRNGKey(0))

In [45]:
params = model.init(key2, X)

(256, 32, 32, 16)
(256, 16, 16, 32)
(256, 8, 8, 32)
(256, 8, 8, 32)


In [46]:
apply_model = jax.jit(model.apply)

In [47]:
apply_model(params, X).shape

(256, 32, 32, 16)
(256, 16, 16, 32)
(256, 8, 8, 32)
(256, 8, 8, 32)


(256, 256)

In [49]:
%timeit apply_model(params, X).shape

4.02 ms ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
VmapImpala = nn.vmap(Impala,
                          variable_axes={'params': 0},
                          split_rngs={'params': True},
                          in_axes=None,
                          out_axes=0,
                          axis_size=10)
ensemble_model = VmapImpala('critic')

In [56]:
ensemble_params = ensemble_model.init(key2, X)

(256, 32, 32, 16)
(256, 16, 16, 32)
(256, 8, 8, 32)
(256, 8, 8, 32)


In [57]:
apply_ensemble = jax.jit(ensemble_model.apply)

In [58]:
apply_ensemble(ensemble_params, X).shape

(256, 32, 32, 16)
(256, 16, 16, 32)
(256, 8, 8, 32)
(256, 8, 8, 32)


(10, 256, 256)

In [59]:
%timeit apply_ensemble(ensemble_params, X).shape

41.2 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
